In [1]:
#libraries
from chemdataextractor import Document
from chemdataextractor.model import Compound,BaseModel
from chemdataextractor.doc import Paragraph, Heading,Sentence
from pprint import pprint

In [2]:
#working temperature property model
from chemdataextractor.model.units import TemperatureModel, Temperature, Kelvin
from chemdataextractor.model import ListType, ModelType, StringType, Compound
from chemdataextractor.parse import I, AutoSentenceParser, join

class WorkingTemperature(TemperatureModel):
    value = StringType()
    units = StringType(contextual=True)

In [3]:
#grammar rules for identifyimg the the working temperature
import re
from chemdataextractor.parse import R, I, W, Optional, merge

prefix = (I('Tmax') +I('of') |I('(')+I('Tmax')+I(')')+I('of')|I('operating')+I('temperature')+I('of')|I('characteristic')+I('temperature')+I('To')+I('=')| (R('^\(T.*max\)$', re.I)+ I('of')) |I('pulsed')+ I('mode') +I('at') + I('temperatures') + I('up')+I('to')|I('working')+I('temperature')+I('of')| I('maximum')+ I('operation') + I('temperature') + I('of')| (I('heat')+ I('-')+I('sink')+I('temperature')+ I('of'))).hide()
units = (Optional(W('°')) + Optional(R('^°?[CFK]\.?$')))('units').add_action(merge)
value = R('^\d+(\.\d+)?$')('value')
wt = (prefix + value + units)('wt')

In [4]:
#the logic for parsing the temperature value
from chemdataextractor.parse.base import BaseSentenceParser
from chemdataextractor.utils import first
from lxml import etree

class WtParser(BaseSentenceParser):
    root = wt

    def interpret(self, result, start, end):
        raw_value = first(result.xpath('./value/text()'))
        raw_units = first(result.xpath('./units/text()'))
        working_temp = self.model(raw_value=raw_value,
                    raw_units=raw_units,
                    value=self.extract_value(raw_value),
                    error=self.extract_error(raw_value),
                    units=self.extract_units(raw_units, strict=True))
        cem_el = first(result.xpath('./cem'))
        if cem_el is not None:
            working_temp.compound = Compound()
            working_temp.compound.names = cem_el.xpath('./name/text()')
            working_temp.compound.labels = cem_el.xpath('./label/text()')
        yield working_temp

In [5]:
#adding the defined temperature parser(logic) to the working temperature property model
WorkingTemperature.parsers = [WtParser()]

In [6]:
#sample sentences from the qcl journals
s1=  Sentence('A large current density dynamic range is observed, leading to a maximum operation temperature of 150 K for the double metal waveguide device')
s2 = Sentence('The laser operated up to a heat-sink temperature of 110K in pulsed mode, 95K in continuous wave (cw) mode, and the threshold current density at 5K was ∼140A∕cm2.')
s3 = Sentence('In pulsed operation, the device lased up to a maximum heat-sink temperature (Tmax) of 111K. The threshold current density (Jth) at 6K was 142A∕cm2. ')

In [14]:
#specifying the model to be used for extracting the temperature values
s1.models = [WorkingTemperature]
s2.models = [WorkingTemperature]
s3.models = [WorkingTemperature] 

In [8]:
pprint(s1.records.serialize())

[{'WorkingTemperature': {'raw_units': 'K',
                         'raw_value': '150',
                         'units': 'Kelvin^(1.0)',
                         'value': '[150.0]'}}]


In [9]:
pprint(s2.records.serialize())

[{'WorkingTemperature': {'raw_units': 'K',
                         'raw_value': '110',
                         'units': 'Kelvin^(1.0)',
                         'value': '[110.0]'}}]


In [10]:
pprint(s3.records.serialize())

[{'WorkingTemperature': {'raw_units': 'K.',
                         'raw_value': '111',
                         'units': 'Kelvin^(1.0)',
                         'value': '[111.0]'}}]


In [11]:
#reading a sample journal paper
with open('345.html', 'rb') as f:
    doc = Document.from_file(f)
print(doc)

<Document: 204 elements>


In [12]:
#Document Metadata
doc.metadata

{'title': '3.4-THz quantum cascade laser based on longitudinal-optical-phonon scattering for depopulation', 'authors': ['Benjamin S.  Williams', 'Hans  Callebaut', 'Sushil  Kumar', 'Qing  Hu', 'John L.  Reno'], 'publisher': 'American Institute of PhysicsAIP', 'journal': 'Applied Physics Letters', 'date': '2003-02-17', 'volume': '82', 'issue': '7', 'firstpage': '1015', 'doi': '10.1063/1.1554479', 'pdf_url': 'https://aip.scitation.org/doi/pdf/10.1063/1.1554479', 'html_url': 'https://aip.scitation.org/doi/abs/10.1063/1.1554479'}

In [13]:
#Extracting the temperature property from the document
doc.models = [WorkingTemperature]
results=doc.records.serialize()
pprint(results)

[{'WorkingTemperature': {'raw_units': 'K',
                         'raw_value': '65',
                         'units': 'Kelvin^(1.0)',
                         'value': '[65.0]'}}]
